In [1]:
from torchvision import datasets, transforms

/Users/leo/miniconda3/envs/dropout-diffusion/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

In [3]:
def transform(img: torch.Tensor):
    return img * 2 - 1.0

train_dataset = datasets.CIFAR10(root="cifar_train", train=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transform,
                                ]),
                                download=True
                                )
val_dataset = datasets.CIFAR10(root="cifar_val", train=False, 
                            transform=transforms.Compose([
                            # transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            transform,
                            ]),
                            download=True)

100%|██████████| 170498071/170498071 [03:18<00:00, 857058.33it/s] 


Extracting cifar_train/cifar-10-python.tar.gz to cifar_train


100%|██████████| 170498071/170498071 [01:00<00:00, 2825335.06it/s]


Extracting cifar_val/cifar-10-python.tar.gz to cifar_val


In [7]:
train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=len(train_dataset), shuffle=False, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=len(val_dataset), shuffle=False, pin_memory=True)

for data in val_loader:
    fit_data = data[0].reshape(10000, -1).detach().cpu().numpy()
    fit_label = data[1].detach().cpu().numpy()

In [8]:
cluster = []
for i in range(10):
    cluster_data = torch.Tensor(fit_data[fit_label==i])
    centroid = torch.mean(cluster_data, dim=0)
    distance = torch.nn.functional.pdist(cluster_data, p=2)
    diameter = torch.max(distance)
    cluster.append({"centroid": centroid, "diameter": diameter})

In [9]:
torch.save(cluster, "cluster_info.pth")

In [28]:
!python lipschitz_base.py --model 1 --model_type 0 --checkpoint "/Users/leo/Downloads/model_best.pth (2).tar"

In [29]:
test_lipschitz = torch.load("lipschitz.pth")

In [18]:
for data in train_loader:
    train_data = data[0]
    train_label = data[1]

In [19]:
from tqdm import tqdm

In [20]:
count_ = [0, 0, 0, 0, 0, 0, 0 ,0, 0 ,0]
for each in tqdm(train_data):
    temp_each = [each.view(-1)]*10
    distance = torch.stack(temp_each) - torch.stack([each['centroid'] for each in cluster])
    #print(distance)
    distance = torch.norm(distance, p=2, dim=1)
    label = torch.argmin(distance)
    #print(distance.shape)
    count_[label.item()] += 1

100%|██████████| 50000/50000 [00:01<00:00, 36885.81it/s]


In [21]:
count_

[9367, 3635, 1690, 1476, 3875, 5315, 11534, 2739, 4270, 6099]

In [30]:
for i in range(10):
    cluster[i]['m'] = count_[i]
    cluster[i]['lipschitz'] = test_lipschitz[i]['lipschitz_const_2']

In [23]:
lipschitz = [each["diameter"]*each["lipschitz"]*each["m"] for each in cluster]

In [31]:
lipschitz = ["{0:.3f}".format(each["lipschitz"]) for each in cluster]

In [32]:
print(" & ".join(lipschitz))

0.200 & 0.141 & 0.134 & 0.157 & 0.156 & 0.148 & 0.118 & 0.150 & 0.160 & 0.149
